#### About
Meanshift Algorithm in Python for Object Tracking

> Object tracking is the task of detecting a moving object in a frame over time.

> About the algorithm
1. Assume a set of points C1, C2 depicting histpgram or pixel density with a rectangular or circular window around it.
2. The task is to move the window around area of maximum pixel density.
3. We pass the initial location of our target object and histogram backprojected to meanshift function.
4. As object translates, The histogram backprojected image changes.
5. Meanshift function moves the window to new location with maximum probability density.

> Assumptions
1. Initial position of the object of interest should be passed to the algorithm.

> Limitations
1. Size of ROI tracker is not adaptive.


In [ ]:
#importing modules
import numpy as np
import cv2 
cap = cv2.VideoCapture('/path/to/video.mp4') #video location
#reading first frame
ret,frame = cap.read()
#ROI for initial pos
x,y,w, h = 240,245,123,75 # initialise as per original values
tracker_roi = (x,y,w,h)
#initialising the roi
roi = frame[y:y+h,x:x+w]
#histogram backprojection shall create image of same size but single size
# 1. Converting to HSV colorspace
convt_image = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)
# 2. Creating mask - discarding low illumination values
mask = cv2.inRange(convt_image, np.array((0.,90.,32.),np.array(180.,255.,255.)))
# 3. Calculating histogram
histogram = cv2.calcHist([convt_image],[0], mask, [180],[0,180]) # using just the HUE channel. hist size-  to 179
# 4. Normalising values between 0 to 255
cv2.normalize(histogram,histogram,alpha=0,beta=255,cv2.NORM_MINMAX)
#5. defining terminating criterion
terminating_criterion = (cv2.TERM_CRITERIA_EPS | cv2.TermCriteria_COUNT,20,1) #either for 20 pixels or by 1 pixel
while(1):
    ret, frame = cap.read()
    if ret==True:
        #calculate hsv value of frame
        convt_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        #calculating backproject
        backproj_image  = cv2.calcBackProject([convt_frame],[0],histogram,[0,180],1)
        #applying meanshift to get new position
        ret, tracker_roi = cv2.meanShift(backproj_image,tracker_roi,terminating_criterion)
        #drawing roi on image
        x,y,w,h = tracker_roi
        out_image = cv2.rectangle(frame,(x,y),(x+w,y+h),255,2,)
        cv2.imshow('Tracked ROI',out_image)
        #optional to view backproj_image
        #cv2.imshow('Backproj_image',backproj_image)
        k = cv2.WaitKey(20)
        if k ==27:
            break
    else:
        break